<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Gonzalo Claro
- Nombre de alumno 2: Camilo Meneses


### **Link de repositorio de GitHub:** https://github.com/GonzaloClaro/evaluaciones_mds7202_grupo23

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 4.8 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 26.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=d2a36277f6bf427a46d82f2afbac839216d7545e9b3c815ddc2127c0b7442856
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=a6253b22c5e89d484061d913f2ef69fd79594e6b38b72e47cd37b500cdf27646
  Stored in directory: /root/.cache/pip/wheels/7f/2a/

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Universidad /2022-1/Lab programación científica/Lab 8/'

In [4]:
df_comics = pd.read_csv(path+'df_comics.csv')
df_comics_no_label = pd.read_csv(path+'comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [6]:
nltk.download('punkt')

docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [7]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Ste mming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [8]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [11]:
bow = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1, 2))

atributos_de_interes = ["intelligence_score","strength_score","speed_score","durability_score","power_score","combat_score"]

preprocessing = ColumnTransformer(
    transformers=[
        ("bow", bow, "history_text"),
        ("MinMaxScaler", MinMaxScaler(), atributos_de_interes),
    ]
)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [12]:
pipeline = Pipeline(
    [
        ("preprosessing", preprocessing),
        ("select", SelectPercentile(score_func=f_classif, percentile=90)),
        ("Classifier", MultinomialNB()),
    ]
)

df_y = df_comics.loc[:,"alignment"]
df_X = df_comics.drop(["alignment"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.33, random_state=42, stratify=df_y 
)

MNB = pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print('Resultados Multinomial NB:')
print(classification_report(y_pred, y_test))

#comparamos con un DummyClassifier
from sklearn.dummy import DummyClassifier

pipeline_DC = Pipeline(
    [
        ("preprosessing", preprocessing),
        ("select", SelectPercentile(score_func=f_classif, percentile=90)),
        ("Classifier", DummyClassifier(strategy = "stratified", random_state = 0)),
    ]
)

DC = pipeline_DC.fit(X_train, y_train)
y_pred_DC = pipeline_DC.predict(X_test)
print('Resultados DummyClassifier:')
print(classification_report(y_test, y_pred_DC))

Resultados Multinomial NB:
              precision    recall  f1-score   support

         Bad       0.12      0.68      0.20        25
        Good       0.98      0.60      0.75       400
     Neutral       0.00      0.00      0.00         0

    accuracy                           0.61       425
   macro avg       0.37      0.43      0.32       425
weighted avg       0.93      0.61      0.72       425



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Resultados DummyClassifier:
              precision    recall  f1-score   support

         Bad       0.33      0.30      0.32       142
        Good       0.56      0.59      0.57       246
     Neutral       0.10      0.11      0.11        37

    accuracy                           0.45       425
   macro avg       0.33      0.33      0.33       425
weighted avg       0.45      0.45      0.45       425



```
Escriba aquí su justificación
```

MultinomialNB: Los resultados son malos sobretodo en la clasificación de las clases que tienen una menor proporción de datos (Bad y Neutral), ya que se sobreajusta a la clase predominante (Good). 

DummyClassifier: Al ser un clasificador al azar no tiene en cuenta ni la cantidad de caracterísicas de cada dato ni los valores de estas, sólo considera las probabilidades a priori de cada clase mediante las distribuciones multinomiales de estas.

1. ¿Nos recomendaría predecir la alineación de BatCow con este clasificador (MultinomialNB)? R: No, ya que se suele ajustar demasiado a la clase mayoritaria, perjudicando la clasificación. En particular no es capaz de acertar a ningun dato de la clase Neutral, por lo que es inviable su uso.

2. Compare el modelo entrenado con un modelo Dummy estratificado. ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar? R: Si, el clasificador Multinomial NB clasifica mejor en cuanto a macro avg en recall. Pero es mejor sólo por poco, además de ser el Dummy un clasificador bastante simple, por lo que no es buen logroy existe un margen de mejora considerable. Como explicamos, MultinomialNB tiende a sobreajustarse a la clase mayoritaria, prediciendola bastante bien a costa de predecir muy mal las otras clases en menor proporción, sobretodo la clase Neutral. 

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [ ]:
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV


selection_pipeline = Pipeline(steps=[
    ("preprocessing", preprocessing),
    ("select", SelectPercentile(score_func=f_classif)),
    ("clf", MultinomialNB()),
])

grid_params = [
    # SVM
    {
        'preprocessing__bow__ngram_range': [(1,1),(1,2),(1,3)],
        'select__percentile': range(20,81,20),
        'clf': [SVC(random_state=0)],
        'clf__kernel': ["linear", "rbf"],
        'preprocessing__bow__max_df': [0.5, 1.0],
        'preprocessing__bow__min_df': [0.0, 0.4]
    },
    {
        'preprocessing__bow__ngram_range': [(1,1),(1,2),(1,3)],
        'select__percentile': range(20,81,20),
        'clf': [LogisticRegression(random_state=0)],
        'clf__C': [1, 0.5, 0.25],
        'preprocessing__bow__max_df': [0.5, 1.0],
        'preprocessing__bow__min_df': [0.0, 0.4]
    },
    {
        'preprocessing__bow__ngram_range': [(1,1),(1,2),(1,3)],
        'select__percentile': range(20,81,20),
        'clf': [RandomForestClassifier(random_state=0)],
        'clf__criterion': ["gini", "entropy","log_loss"],
        'preprocessing__bow__max_df': [0.5, 1.0],
        'preprocessing__bow__min_df': [0.0, 0.4]
    },
]

hgs = HalvingGridSearchCV(selection_pipeline, grid_params, verbose = 10)

hgs.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 30
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 384
n_resources: 30
Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV 1/5; 1/384] START clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20
[CV 1/5; 1/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20;, score=(train=1.000, test=0.167) total time=   1.3s
[CV 2/5; 1/384] START clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20
[CV 2/5; 1/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessi

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 3/5; 1/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20;, score=(train=1.000, test=0.167) total time=   0.7s
[CV 4/5; 1/384] START clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20
[CV 4/5; 1/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20;, score=(train=1.000, test=0.500) total time=   0.3s
[CV 5/5; 1/384] START clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=20
[CV 5/5; 1/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 3/5; 2/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=40;, score=(train=1.000, test=0.167) total time=   0.7s
[CV 4/5; 2/384] START clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=40
[CV 4/5; 2/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=40;, score=(train=1.000, test=0.500) total time=   0.3s
[CV 5/5; 2/384] START clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=40
[CV 5/5; 2/384] END clf=SVC(random_state=0), clf__kernel=linear, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.

...

[CV 5/5; 75/128] END clf=LogisticRegression(random_state=0), clf__C=1, preprocessing__bow__max_df=1.0, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 2), select__percentile=20;, score=(train=0.667, test=0.667) total time=   1.9s
[CV 1/5; 76/128] START clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 3), select__percentile=40
[CV 1/5; 76/128] END clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 3), select__percentile=40;, score=(train=0.750, test=0.556) total time=   2.7s
[CV 2/5; 76/128] START clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 3), select__percentile=40
[CV 2/5; 76/128] END clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 4/5; 76/128] END clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 3), select__percentile=40;, score=(train=0.667, test=0.667) total time=   3.8s
[CV 5/5; 76/128] START clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 3), select__percentile=40
[CV 5/5; 76/128] END clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 3), select__percentile=40;, score=(train=0.694, test=0.556) total time=   2.7s
[CV 1/5; 77/128] START clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.4, preprocessing__bow__ngram_range=(1, 1), select__percentile=60
[CV 1/5; 77/128] END clf=LogisticRegression(random_state=0), clf__C=0.25, preprocessing__b

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 640.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-pac

[CV 1/5; 1/43] END clf=RandomForestClassifier(random_state=0), clf__criterion=log_loss, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=60;, score=(train=nan, test=nan) total time=   5.5s
[CV 2/5; 1/43] START clf=RandomForestClassifier(random_state=0), clf__criterion=log_loss, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=60
[CV 2/5; 1/43] END clf=RandomForestClassifier(random_state=0), clf__criterion=log_loss, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=60;, score=(train=nan, test=nan) total time=   5.2s
[CV 3/5; 1/43] START clf=RandomForestClassifier(random_state=0), clf__criterion=log_loss, preprocessing__bow__max_df=0.5, preprocessing__bow__min_df=0.0, preprocessing__bow__ngram_range=(1, 1), select__percentile=60
[CV 3/5; 1/43] END clf=RandomForestClassif

In [ ]:
print(hgs.best_score_)
print(hgs.best_params_)

In [19]:
selection_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessing', 'select', 'clf', 'preprocessing__n_jobs', 'preprocessing__remainder', 'preprocessing__sparse_threshold', 'preprocessing__transformer_weights', 'preprocessing__transformers', 'preprocessing__verbose', 'preprocessing__verbose_feature_names_out', 'preprocessing__bow', 'preprocessing__MinMaxScaler', 'preprocessing__bow__analyzer', 'preprocessing__bow__binary', 'preprocessing__bow__decode_error', 'preprocessing__bow__dtype', 'preprocessing__bow__encoding', 'preprocessing__bow__input', 'preprocessing__bow__lowercase', 'preprocessing__bow__max_df', 'preprocessing__bow__max_features', 'preprocessing__bow__min_df', 'preprocessing__bow__ngram_range', 'preprocessing__bow__preprocessor', 'preprocessing__bow__stop_words', 'preprocessing__bow__strip_accents', 'preprocessing__bow__token_pattern', 'preprocessing__bow__tokenizer', 'preprocessing__bow__vocabulary', 'preprocessing__MinMaxScaler__clip', 'preprocessing__MinMaxScaler__copy', 'preproc

```
Escriba aquí su justificación
```

En una primera instancia, la mejor combinación encontrada está dada por el clasificador LogisticRegression, con parámetros C=0.5 y percentil 60. Lo que sucedió fue que se volvió a intentar con otra grilla y no se alcanzó a ejecutar el código a tiempo.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [ ]:
#eliminamos los nan
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) # eliminar ejemplos sin historia
y = df_comics_no_label.loc[:,"alignment"]
X = df_comics_no_label.drop(["alignment"], axis=1)

In [ ]:
atributos_de_interes = [
    "intelligence_score",
    "strength_score",
    "speed_score",
    "durability_score",
    "power_score",
    "combat_score",
]

bow = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1, 1))

preprocessing = ColumnTransformer(
    transformers=[
        ("bow", bow, "history_text"),
        ("MinMaxScaler", MinMaxScaler(), atributos_de_interes),
    ]
)

#utilizamos el mejor clasificador para el pipeline
pipeline_optimo = Pipeline(
    [
        ("preprosessing", preprocessing),
        ("select", SelectPercentile(score_func=f_classif, percentile=60)),
        ("Classifier", LogisticRegression(C=0.5, random_state=0)),
    ]
)

pipeline_optimo.fit(df_X,df_y)

y_pred = pipeline_optimo.predict(X)
df_comics_no_label['alignment'] = y_pred
df_comics_no_label.loc[df_comics_no_label['name'].isin(['Vergil', 'Gorilla Girl', 'Batcow']),['name','alignment']]

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>